In [ ]:
from pydub import AudioSegment
import numpy as np
import os
import librosa
import torch
import noisereduce as nr
import datetime

hf_token = "hf_token"

In [ ]:
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True)
model.to(device)
processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device)

In [35]:
def extract_audio(video_path:str, output_audio_path="temp_audio.wav"):
    "xtracts audio.wav frtom video file"
    try:
        video = AudioSegment.from_file(video_path)
        audio = video.split_to_mono()[0]  
        audio.export(output_audio_path, format="wav")
        return output_audio_path
    
    except Exception as e:
        print(str(e))


def speakers_with_timestamp(audio_path:str):
    "produces timestamped speaker information from audio file"
    from pyannote.audio import Pipeline
    pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization-3.1", use_auth_token=hf_token)
    diarization = pipeline(audio_path)
    speaker_timestamps=[]
    for turn, _, speaker in diarization.itertracks(yield_label=True):
        speaker_timestamps.append({'speaker':speaker,'start':round(turn.start,2),'end':round(turn.end,2)})
    return speaker_timestamps


def preprocess_(audio_path:str,speaker_timestamps:list):
    "creates a dataset for transcribing the spoken parts based on the speaker timestamps"
    y,sr=librosa.load(audio_path)
    y=y.astype(np.float64)
    y = nr.reduce_noise(y=y, sr=sr) 
    y = y / np.max(np.abs(y))
    audio_samples=[]
    for item in speaker_timestamps:
        sample=y[round(item['start']*sr):round(item['end']*sr)+1]
        audio_samples.append({"array":sample,'sampling_rate':sr})
    return audio_samples


def transcrsibe(video_path:str,language:str,translate:bool):
    """adds transcribed text information to the speaker with timestamp
    video_path: input video path
    translate: translates to english if set to True else in - language
    language: language to trancribe in
    """
    audio_path=extract_audio(video_path)
    sp_timstamp=speakers_with_timestamp(audio_path)
    samples_=preprocess_(audio_path,sp_timstamp)
    for idx,sample_ in enumerate(samples_):
        generate_kwargs={"language": language,"task": "translate"} if translate  else {"language": language}
        result = pipe(sample_,return_timestamps=True,generate_kwargs=generate_kwargs)
        sp_timstamp[idx].update({'text':result['text']})
    return sp_timstamp


def format_time(seconds):
    "time formatting for srt"
    td = datetime.timedelta(seconds=seconds)
    hrs, remainder = divmod(td.seconds, 3600)
    mins, secs = divmod(remainder, 60)
    millis = td.microseconds // 1000
    return f"{hrs:02}:{mins:02}:{secs:02},{millis:03}"



def out_file(data,name):
    "writes the informatison into srt file and saves it"
    with open(name, 'w') as file:
        for idx, item in enumerate(data, start=1):
            start_time = format_time(item['start'])
            end_time = format_time(item['end'])
            file.write(f"{idx}\n{start_time} --> {end_time}\n{item['speaker']}: {item['text']}\n\n")

In [37]:
%%time

# running inference

video_path='/kaggle/input/video-examples-data/hindi_multi.mp4'  #any input video path
out=transcrsibe(video_path=video_path,language='hindi',translate=True)

/usr/local/lib/python3.10/dist-packages/pyannote/audio/models/blocks/pooling.py:104: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at ../aten/src/ATen/native/ReduceOps.cpp:1823.)
  std = sequences.std(dim=-1, correction=1)
/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:512: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


CPU times: user 1min 42s, sys: 1min 16s, total: 2min 59s
Wall time: 1min 37s


In [38]:

# saving the file and reding it in to seee the out

out_file(out,'transcript.srt')

with open('/kaggle/working/transcript.srt', 'r') as file:
    subtitles = file.read()
print(subtitles)

1
00:00:00,080 --> 00:00:01,350
SPEAKER_00:  Ravi, bring me a cup of coffee.

2
00:00:01,870 --> 00:00:02,260
SPEAKER_00:  So, what?

3
00:00:03,860 --> 00:00:04,170
SPEAKER_01:  No.

4
00:00:04,920 --> 00:00:05,630
SPEAKER_01:  Hey Shetty!

5
00:00:07,050 --> 00:00:08,100
SPEAKER_01:  You have two options.

6
00:00:09,030 --> 00:00:09,720
SPEAKER_01:  Leave us.

7
00:00:10,800 --> 00:00:13,780
SPEAKER_01:  You'll get so much money that you won't have a place to stay in this police station.

8
00:00:15,200 --> 00:00:15,220
SPEAKER_01:  Welcome to our Channel.

9
00:00:15,220 --> 00:00:15,250
SPEAKER_02:  Welcome to our Channel.

10
00:00:15,250 --> 00:00:16,160
SPEAKER_01:  And the second option

11
00:00:16,160 --> 00:00:16,180
SPEAKER_02:  Welcome to our Channel.

12
00:00:17,010 --> 00:00:17,020
SPEAKER_01:  Thank you for watching.

13
00:00:17,020 --> 00:00:18,580
SPEAKER_02:  You won't be able to save this police station.

14
00:00:20,480 --> 00:00:21,510
SPEAKER_02:  You don't kn